In [9]:
from http.server import SimpleHTTPRequestHandler, HTTPServer
from urllib import request, error
import pandas as pd
from pycaret.clustering import *
import urllib.parse
import sys

badwords = ['sleep', 'drop', 'uid', 'select', 'waitfor', 'dealy', 'system', 'system', 'union', 'order by', 'group by']

def ExtractFeatures(path):
    path = urllib.parse.unquote(path)
    badwords_count = 0
    single_q = path.count("'")
    double_q = path.count("\"")
    dashes = path.count("--")
    braces = path.count("(")
    spaces = path.count(" ")
    for word in badwords:
        badwords_count += path.count(word)
    lst = [single_q, double_q, dashes, braces, spaces, badwords_count]
    print(lst)
    return pd.DataFrame([lst], columns=['single_q', 'double_q', 'dashes', 'braces', 'spaces', 'badwords_count'])


In [10]:
http=pd.read_csv(r'C:\Users\tusha\Documents\minor project\httplog.csv')
clu1 = setup(data=http, normalize=True, numeric_features=['single_q', 'double_q', 'dashes', 'braces', 'spaces', 'badwords'], ignore_features=['method', 'path', 'body', 'class'])
kmeans=create_model('kmeans',num_clusters=2)

,Description,Value
0,Session id,4754
1,Original data shape,"(275, 11)"
2,Transformed data shape,"(275, 86)"
3,Ignore features,4
4,Numeric features,6
5,Categorical features,1
6,Rows with missing values,0.4%
7,Preprocess,True
8,Imputation type,simple
9,Numeric imputation,mean


,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.6906,12.7035,0.2067,0,0,0


Processing:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
class SimpleHTTPProxy(SimpleHTTPRequestHandler):
    proxy_routes = {}

    @classmethod
    def set_routes(cls, proxy_routes):
        cls.proxy_routes = proxy_routes

    def do_GET(self):
        parts = self.path.split('/')
        print(parts)
        live_data = ExtractFeatures(parts[3])
        result = predict_model(kmeans, data=live_data)
        print(result['Cluster'][0])
        if result['Cluster'][0] == "Cluster 1":
            print('Intrusion detected')
        if len(parts) >= 2:
            self.proxy_request('http://' + parts[2] + '/')
        else:
            super().do_GET()

    def proxy_request(self, url):
        try:
            response = request.urlopen(url)
        except error.HTTPError as e:
            print('err')
            self.send_response_only(e.code)
            self.end_header()
            return
        self.send_response_only(response.status)
        for name, value in response.headers.items():
            self.send_header(name, value)
        self.end_headers()
        self.copyfile(response, self.wfile)

SimpleHTTPProxy.set_routes({'proxy_route': 'http://demo.testfire.net/'})
with HTTPServer(('127.0.0.1', 8081), SimpleHTTPProxy) as httpd:
    host, port = httpd.socket.getsockname()
    print(f'Listening on http://{host}:{port}')
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        print("\nKeyboard interrupt received, exiting.")
        sys.exit(0)